In [3]:
import pandas as pd
# from io import StringIO

df = pd.read_csv('decN_melted.csv')

# Calculate the probability of each LOS grouped by 'Local Date' and 'Time'
los_counts = df.groupby(['Local Date', 'Time', 'LOS']).size().unstack(fill_value=0)
los_probabilities = los_counts.div(los_counts.sum(axis=1), axis=0)

los_probabilities.tail()  # Show top rows of the probabilities


LOS                    A    B    C    E    F
Local Date Time                             
2023-12-31 19:00:00  1.0  0.0  0.0  0.0  0.0
           20:00:00  1.0  0.0  0.0  0.0  0.0
           21:00:00  1.0  0.0  0.0  0.0  0.0
           22:00:00  1.0  0.0  0.0  0.0  0.0
           23:00:00  1.0  0.0  0.0  0.0  0.0

In [4]:
# Ensure 'Time' column is in the correct format if necessary
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Initialize an empty list to store results
results = []

# Loop through each unique time in the 'Time' column
for specific_time in df['Time'].unique():
    # Filter the data for the specific time
    filtered_df = df[df['Time'] == specific_time]

    # Count occurrences of each LOS for the specific time
    los_counts = filtered_df['LOS'].value_counts()

    # Calculate probabilities if there are counts
    if not los_counts.empty:
        total = los_counts.sum()
        los_probabilities = los_counts / total

        # Append each probability result as a dictionary to the results list
        for los, probability in los_probabilities.items():
            results.append({'Time': specific_time, 'LOS': los, 'Probability': probability})
    else:
        # Append zero probabilities for times without data
        results.append({'Time': specific_time, 'LOS': None, 'Probability': 0})

# Convert results list to a DataFrame
probabilities_df = pd.DataFrame(results)

# Display the final DataFrame
print(probabilities_df)

output_file_path = 'probabilities_df.csv'
probabilities_df.to_csv(output_file_path, index=False)


        Time LOS  Probability
0   00:00:00   A     1.000000
1   01:00:00   A     1.000000
2   02:00:00   A     1.000000
3   03:00:00   A     1.000000
4   04:00:00   A     1.000000
5   05:00:00   A     1.000000
6   06:00:00   A     1.000000
7   07:00:00   A     0.483871
8   07:00:00   C     0.387097
9   07:00:00   B     0.129032
10  08:00:00   C     0.451613
11  08:00:00   A     0.419355
12  08:00:00   B     0.129032
13  09:00:00   C     0.451613
14  09:00:00   B     0.387097
15  09:00:00   A     0.161290
16  10:00:00   C     0.548387
17  10:00:00   B     0.419355
18  10:00:00   A     0.032258
19  11:00:00   C     0.612903
20  11:00:00   B     0.322581
21  11:00:00   A     0.032258
22  11:00:00   E     0.032258
23  12:00:00   C     0.548387
24  12:00:00   B     0.387097
25  12:00:00   A     0.032258
26  12:00:00   F     0.032258
27  13:00:00   B     0.709677
28  13:00:00   C     0.225806
29  13:00:00   A     0.032258
30  13:00:00   F     0.032258
31  14:00:00   B     0.709677
32  14:00:

In [5]:
def parse_dates(date_series):
    try:
        # Try parsing with dayfirst=True
        return pd.to_datetime(date_series, dayfirst=True)
    except ValueError:
        # If it fails, try parsing without additional arguments
        return pd.to_datetime(date_series)


# Convert 'Local Date' to datetime
df['Local Date'] = parse_dates(df['Local Date'])

# Initialize an empty list to store results
results = []

# Loop through each unique date in the 'Local Date' column
for specific_date in df['Local Date'].unique():
    # Filter the data for the specific date
    filtered_df = df[df['Local Date'] == specific_date]

    # Count occurrences of each LOS for the specific date
    los_counts = filtered_df['LOS'].value_counts()

    # Calculate probabilities if there are counts
    if not los_counts.empty:
        total = los_counts.sum()
        los_probabilities = los_counts / total

        # Append each probability result as a dictionary to the results list
        for los, probability in los_probabilities.items():
            results.append({'Date': specific_date, 'LOS': los, 'Probability': probability})
    else:
        # Append zero probabilities for dates without data
        results.append({'Date': specific_date, 'LOS': None, 'Probability': 0})

# Convert results list to a DataFrame
probabilities_df = pd.DataFrame(results)

# Display the final DataFrame
print(probabilities_df)

output_file_path = 'probabilities_df.csv'
probabilities_df.to_csv(output_file_path, index=False)


         Date LOS  Probability
0  2023-12-01   A     0.500000
1  2023-12-01   C     0.375000
2  2023-12-01   B     0.125000
3  2023-12-02   A     0.666667
4  2023-12-02   B     0.208333
..        ...  ..          ...
85 2023-12-30   C     0.291667
86 2023-12-30   B     0.041667
87 2023-12-31   A     0.708333
88 2023-12-31   B     0.208333
89 2023-12-31   C     0.083333

[90 rows x 3 columns]


In [6]:
df['Local Date'] = parse_dates(df['Local Date'])

# # Calculate the probability of each LOS grouped by 'Local Date' and 'Time'
los_counts = df.groupby(['Local Date', 'Time', 'LOS']).size().unstack(fill_value=0)
los_probabilities = los_counts.div(los_counts.sum(axis=1), axis=0)

# Add weekday and week number columns
df['Weekday'] = df['Local Date'].dt.day_name()  # Extract the day name
df['Week'] = df['Local Date'].dt.isocalendar().week  # Extract the week number
df['Month'] = df['Local Date'].dt.month  # Extract the month

# Calculate probabilities grouped by weekday
los_counts_weekday = df.groupby(['Weekday', 'LOS']).size().unstack(fill_value=0)
los_probabilities_weekday = los_counts_weekday.div(los_counts_weekday.sum(axis=1), axis=0)

print("\nProbability of each LOS by weekday:")
print(los_probabilities_weekday)

# Calculate probabilities grouped by week
los_counts_week = df.groupby(['Week', 'LOS']).size().unstack(fill_value=0)
los_probabilities_week = los_counts_week.div(los_counts_week.sum(axis=1), axis=0)

print("\nProbability of each LOS by week:")
print(los_probabilities_week)

# Calculate probabilities grouped by month
los_counts_month = df.groupby(['Month', 'LOS']).size().unstack(fill_value=0)
los_probabilities_month = los_counts_month.div(los_counts_month.sum(axis=1), axis=0)

print("\nProbability of each LOS by month:")
print(los_probabilities_month)




Probability of each LOS by weekday:
LOS               A         B         C         E         F
Weekday                                                    
Friday     0.516667  0.125000  0.316667  0.008333  0.033333
Monday     0.645833  0.166667  0.187500  0.000000  0.000000
Saturday   0.633333  0.216667  0.150000  0.000000  0.000000
Sunday     0.683333  0.283333  0.033333  0.000000  0.000000
Thursday   0.531250  0.302083  0.166667  0.000000  0.000000
Tuesday    0.572917  0.302083  0.125000  0.000000  0.000000
Wednesday  0.552083  0.218750  0.229167  0.000000  0.000000

Probability of each LOS by week:
LOS          A         B         C         E        F
Week                                                 
48    0.625000  0.208333  0.166667  0.000000  0.00000
49    0.541667  0.261905  0.196429  0.000000  0.00000
50    0.535714  0.255952  0.208333  0.000000  0.00000
51    0.577381  0.285714  0.136905  0.000000  0.00000
52    0.702381  0.119048  0.148810  0.005952  0.02381

Probabilit